In [349]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import codecs
import os
import gzip
import sys
from bs4 import BeautifulSoup
from itertools import chain        
from contextlib import ExitStack 
import time
import json
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import pymorphy2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from pytorch_transformers import BertTokenizer, BertConfig
from transformers import  BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [390]:
morph = pymorphy2.MorphAnalyzer()

In [391]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [392]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [393]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [395]:
stop_words = set(stopwords.words(["russian", "english"]))

In [396]:
lock = Lock()

In [398]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [399]:
def Parser_for_titles_without_stemming(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [word for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [400]:
def Parser_for_text(ID):
    with lock:
        text_id = df[df.id == ID].text.values
    text_id = text_id[0]
    stemmed_text = [stemmerR.stem(stemmerE.stem(word)) for word in text_id.split(' ')[:512]]
    with lock:
        vector_of_text[str(ID)] = ' '.join([word for word in stemmed_text])

In [120]:
df_iter = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/data/project.csv', encoding='utf-8', chunksize=500)

In [159]:
i = 0
for df in df_iter:
    df.fillna(' ')
    vector_of_text = {}
    idx = df.id.values
    with ThreadPool(10) as pool:
        pool.map(Parser_for_text, idx)
    with open('texts/text_stemmed_part_{:05d}.json'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        record = json.dumps(vector_of_text, ensure_ascii=False)
        print(record, file=f_json)
    i += 1

есть 57 файлов с индексами страниц по 500

In [401]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles_without_stemming, list(range(1,28027)))

In [403]:
with open('titles_without_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [2]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model

Взяли заголовки

In [3]:
vec = {}

In [4]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [5]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [6]:
model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')

In [7]:
len(sentences)

28026

In [8]:
first = sentences[0]
first

'м б аншин центр репродукц и генетик фертимед г москв'

In [9]:
len(model_fasttext.wv[first])

100

Давайте попробуем сделать общий файл с эмеддингами текстов.. не просто так же их парсили

Делаем эмбединги и запоминаем их

In [40]:
filenames = os.listdir('texts_57_json')
texts_embeddings = {}
texts_embeding_to_json = {}
for f_name in filenames:
    file_n = 'texts_57_json/' + f_name
    with open(file_n, mode='r') as file:
        d = json.load(file)  # все тексты 
        for key, val in d.items():
            embedding = model_fasttext.wv[val]
            embedding_string = tuple(map(str, embedding))
            texts_embeddings[key] = embedding
            texts_embeding_to_json[key] = embedding_string

Хочу сделать Tf-Idf векторизацию

Запись в файл стринговых эмбеддингов

In [43]:
with open('texts_embeddings.json', 'w', encoding='utf8') as file:
    json.dump(texts_embeding_to_json, file)

Попробуем прочитать эмбеддинги - не забудь перевести в float

In [1367]:
with open('texts_embeddings.json', 'r', encoding='utf8') as file:
    texts_embedding_from_json = json.load(file)

In [1368]:
for key, val in texts_embedding_from_json.items():
    embedding = np.array(list(map(float, texts_embedding_from_json[key])))
    texts_embedding_from_json[key] = embedding

## Фичи

Если фасттекст применять к текстам после стемминга, результат получается лучше почему-то.

Для заголовков:
1) Метки кластеров внутри группы
2) Силуэт для объекта группы

In [1372]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('/Users/egor/Desktop/Техносфера/MachineLearning/project/docs_titles.tsv') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
        self.doc_to_vec = TfidfVectorizer().fit_transform([self.doc_to_title[i] for i in self.doc_to_title.keys()])
        self.top_k_cosine_titles = 15 # k ближайших косинусных расстояний для заголовков
        #self.top_k_cosine_texts = 20 # k ближайших косинусных расстояний для заголовков
    
    def make_train(self):
        train_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/train_groups.csv')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.zeros((train_data.shape[0], self.top_k_cosine_titles)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        rand_index = []
        for group_id, group_indx in groups.groups.items():
            group = train_data.iloc[group_indx]
            docs_id = group.doc_id.values
            # сначала заберем все файлы из группы
            embedded_texts = []
            group_titles = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            targets = group.target.values
            #model_titles = DBSCAN(0.65, metric="cosine", min_samples=8)
            #labels_titles = model_titles.fit_predict(embedded_titles)
            '''
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=4)
            labels_texts = model_texts.fit_predict(embedded_texts)
            '''
            # добавим фичи из расстояний
            # получим матрицу из попарных расстояний
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_titles] = cos_features
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
            #My_X[i: i + group_indx.size, -2] = labels_titles
            #My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
        
        
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/test_groups.csv')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
            
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:25])
        X_test = np.array(X_test)
        
        i = 0
        My_X = np.zeros((test_data.shape[0], self.top_k_cosine_titles)) # кол-во новых фичей
        groups = test_data.groupby('group_id')
        for group_id, group_indx in groups.groups.items():
            group = test_data.iloc[group_indx]
            docs_id = group.doc_id.values
            group_titles = []
            embedded_texts = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            #model_titles = DBSCAN(0.45, metric="cosine", min_samples=8)
            #labels_titles = model_titles.fit_predict(embedded_titles)
            '''
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=4)
            labels_texts = model_texts.fit_predict(embedded_texts)
            '''
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_titles] = cos_features
            #My_X[i: i + group_indx.size, -2] = labels_titles
            #My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
            
            #My_X[i: i + group_indx.size, self.top_k_cosine_titles:self.top_k_cosine_titles
            #    + self.top_k_cosine_texts] = cos_features_texts
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

### Валидация для кластеризации

Валидация для заголовков дала DBSCAN(0.6, metric="cosine", min_samples=8)

In [1373]:
importer = Import()

In [1374]:
X_train, y_train, groups_train = importer.make_train()

In [1375]:
X_test, groups_test, pair_ids = importer.make_test()

In [1376]:
X_train.shape

(11690, 40)

In [1377]:
X_test.shape

(16627, 40)

## Модельки

In [1340]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [1341]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.fit_transform(X_test)

In [1342]:
def feature_generator(X_train, X_test, num_model):
    indices = np.array([])
    length = X_train.shape[1]
    features = np.arange(length)
    np.random.shuffle(features)
    for i in range(num_model):
        ind = np.random.randint(0, length, length)
        ind = np.unique(np.append(ind, features[i * (length // num_model) : (i + 1) * (length // num_model + 1)]))
        yield X_train[:, ind], X_test[:, ind]

In [1343]:
clf_e_1 = RandomForestClassifier(max_depth=10, n_estimators=150)
clf_e_2 = ExtraTreesClassifier(max_depth=15)
clf_e_4 = catboost.CatBoostClassifier()

In [1313]:
res = (clf_e_1.predict_proba(X_train_sk)[:, 1] + clf_e_2.predict_proba(X_train_sk)[: ,1]
+ clf_e_4.predict_proba(X_train_sk)[:, 1])/ 3

In [1314]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res > th, dtype=int)))

0.7080086580086581
0.7842605156037992
0.8378781712115045
0.8570594247335378
0.8558213716108454
0.8244118649846572
0.756608933454877


In [1315]:
f1_score(y_train, np.array(res > 0.4, dtype=int))

0.8570594247335378

Для DBSCAN(0.475, samples = 7):

0.82 при 5 соседях для косинуса

0.848095724500219 при 15 соседях

0.8509769612131818 при 20 соседях

0.8545427858505797 при 25 соседях

Для DBSCAN(0.6, samples = 7):

0.8305378831694621 на 5 соседях

0.8496447730897491 на 15 соседях

0.858557902403496 на 25 соседях

Min_samples = 8 дает получше скор

model_titles = DBSCAN(0.7, metric="cosine", min_samples=8):

На 25 соседях 0.8600523712540006

model_titles = DBSCAN(0.7, metric="cosine", min_samples=7) на 25 соседях дает 0.8593886462882097

model_titles = DBSCAN(0.7, metric="cosine", min_samples=9) на 25 соседях дает 0.8584289240598229

model_titles = DBSCAN(0.65, metric="cosine", min_samples=8) на 25 соседях дает 0.8568099053168244

model_titles = DBSCAN(0.6, metric="cosine", min_samples=8) на 25 соседях дает 0.8567264247194286

##### Добавил соседей по текстам:


DBSCAN(0.7, metric="cosine", min_samples=5)

Для 3 соседей 0.8733713951105255

Для 15 соседей 0.8819013876586951

Для 25 соседей 0.8833702882483371

##### model_titles = DBSCAN(0.65, metric="cosine", min_samples=8) на 25 соседях дает 0.8568099053168244 -  взял это, но количество соседей никак не учитывается

Для 3 соседей model_texts = DBSCAN(0.05, metric="cosine", min_samples=5) дает 0.857598607888631

Для 3 соседей model_texts = DBSCAN(0.15, metric="cosine", min_samples=5) дает 0.8583466511695481 

Для 3 соседей model_texts = DBSCAN(0.2, metric="cosine", min_samples=5) дает 0.8596746077861709

Для 3 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=5) дает 0.8609368159929958

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=5) дает 0.8622248141128445 но это мне не нравится, потому что похоже на переобучение

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=6) дает 0.8609734771203731

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=4) дает 0.8611030055442077

##### Попрубую подключить одно косинусное расстояние для текстов, посмотрим, что будет

Для 1 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.8664050007268499

Для 2 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.8681672025723474

Для 20 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.880437999408109

Я не верю в расстояния между текстами!!!!

In [872]:
test = (clf_e_1.predict_proba(X_test_sk)[:, 1] + clf_e_2.predict_proba(X_test_sk)[: ,1]
+ clf_e_4.predict_proba(X_test_sk)[:, 1])/ 3

In [873]:
y_test = np.array(test > 0.4, dtype=int)

In [874]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_test[i]) + str("\n"))

In [1357]:
X_train[:,-1] == 1

array([False, False, False, ..., False, False, False])

# Уже прям боевая валидация

In [1344]:
kfold = StratifiedKFold(n_splits=5)

In [1345]:
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train_sk,y_train)

ada_best = gsadaDTC.best_estimator_

Fitting 5 folds for each of 112 candidates, totalling 560 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   26.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 560 out of 560 | elapsed:  2.9min finished


In [1346]:
res_ada = ada_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_ada > th, dtype=int)))

0.4466148428675835
0.4466148428675835
0.9997025580011898
0.9997025580011898
0.9997025580011898
0.9997023809523811
0.9997023809523811


In [1347]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train_sk,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   27.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:  4.0min finished


0.7009634518386404

In [1348]:
res_extra = ExtC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_extra > th, dtype=int)))

0.6843609636586362
0.7494666982697322
0.7876939397957831
0.7901701323251419
0.775
0.7386007657500872
0.6559969147705361


In [1349]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train_sk,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   52.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  7.9min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed: 19.9min finished


0.7000899412507076

In [1350]:
res_forest = RFC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_forest > th, dtype=int)))

0.7052775154758157
0.7990367248645395
0.8394004282655246
0.8556566970091027
0.8460931333859512
0.8095238095238096
0.7291007261217649


In [1351]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train_sk,y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.0min


KeyboardInterrupt: 

In [ ]:
res_grad = GBC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_grad > th, dtype=int)))

In [1292]:
# Gradient boosting tunning

XGB = xgb.XGBClassifier()
xgb_param_grid = {'alpha' : [0.01, 0.1],
              'n_estimators' : [10,20,30],
              'eval_metric': ['auc'],
              'max_depth': [4, 6],
              'scale_pos_weight': [1, 2, 2.4] 
              }

gsXGB = GridSearchCV(XGB,param_grid = xgb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsXGB.fit(X_train_sk,y_train)

XGB_best = gsXGB.best_estimator_

# Best score
gsXGB.best_score_

NameError: name 'xgb' is not defined

In [ ]:
res_xgb = XGB_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_xgb > th, dtype=int)))

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
estimators = [('rf', RFC_best), ('et', ExtC_best), ('gb', GBC_best)]

In [ ]:
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [ ]:
stack.fit(X_train_sk,y_train)

In [ ]:
stack_res = stack.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(stack_res > th, dtype=int)))

Нужно подобрать порог!!!

In [ ]:
y_pls = np.array(stack.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

In [1301]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

на Кэгле = 0.68162 - сильное переобучение - решил убрать расстояние между текстами. Было 25 cosine, 0.65, 8; 0.3, 4

на Кэгле = 0.69587 без текстов но 25 cosine, 0.65, 8; 0.3, 4

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best), ('stack', stack)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

In [ ]:
vote = votingC.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(vote > th, dtype=int)))

Валидация RandomForest + ExtraTree = 0.8157055925818604 на 25 фичах на titles и 3 на текстах. model_titles = DBSCAN(0.6, metric="cosine", min_samples=8); model_texts = DBSCAN(0.1, metric="cosine", min_samples=5)

In [1332]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

In [1333]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

на Кэгле 0.69022 текстов но 25 cosine, 0.65, 8; 0.1, 4

У Вани на фичах без текстов на рандомфоресте и экстраТри 0.71 на кагле

У Вани на фичах с 20 текстами на рандомфоресте и экстраТри 0.706 на кагле

Три модельки на маленьком количестве фичей 0.71702

Стекинг на 25 + 20 + 2 + 25 фичей от Севы и три модели  скор = 0.6992. Наверное из-за текстов

Стекинг на маленьком количестве фичей скор = 0.71886

Без стекинга Голосование из 4 моделей: RandomForest, ExtraTrees, Градиентный бустинг, XGBoost 25 + 25 + 2 фичей  скор на кагле = 0.72434

##### Голосование из 5 моделей: Stacking, RandomForest, ExtraTrees, Градиентный бустинг, XGBoost 25 + 25 + 2 фичей скор на кагле = 0.7270

In [242]:
!pip install autokeras

     |████████████████████████████████| 164 kB 975 kB/s eta 0:00:01
     |████████████████████████████████| 62 kB 721 kB/s eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)


  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78936 sha256=5b8eacfd3d09dd47ffe5decadce22a2d57b62f0a46c588730b79fab7b168ed05
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/78/e2/80/7fe373cad54ad22b06d0d6204cbc29cead9e69bb2680327775
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=7c6f55f8556c75aaa3a92189474a1ffb6f2ca6f8c73e8020606f772f5a8b3dda
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=9d9d12e13a5cd421f5588c17da0e5c9c02f47e7ed7c9da558a50df822f5241d9
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner future terminaltables
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m 

In [243]:
import autokeras as ak

In [244]:
X_train.shape

(11690, 36)

In [245]:
y_train.shape

(11690,)

In [246]:
data = pd.DataFrame()

In [253]:
y = np.array(y_train)[:,np.newaxis]
y.shape

(11690, 1)

In [257]:
Train_df = pd.DataFrame(data=X_y)

In [258]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11690 entries, 0 to 11689
Data columns (total 37 columns):
0     11690 non-null float64
1     11690 non-null float64
2     11690 non-null float64
3     11690 non-null float64
4     11690 non-null float64
5     11690 non-null float64
6     11690 non-null float64
7     11690 non-null float64
8     11690 non-null float64
9     11690 non-null float64
10    11690 non-null float64
11    11690 non-null float64
12    11690 non-null float64
13    11690 non-null float64
14    11690 non-null float64
15    11690 non-null float64
16    11690 non-null float64
17    11690 non-null float64
18    11690 non-null float64
19    11690 non-null float64
20    11690 non-null float64
21    11690 non-null float64
22    11690 non-null float64
23    11690 non-null float64
24    11690 non-null float64
25    11690 non-null float64
26    11690 non-null float64
27    11690 non-null float64
28    11690 non-null float64
29    11690 non-null float64
30    11690 non-null 

In [259]:
Train_df.to_csv('train_df.csv')

In [263]:
Train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,4.0,4.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,...,0.614383,0.528873,0.509626,0.504905,0.501869,0.461926,0.449864,-1.0,0.0,0.0
1,8.0,7.0,7.0,7.0,7.0,7.0,4.0,4.0,3.0,3.0,...,0.653389,0.628512,0.580146,0.533492,0.457677,0.426910,0.407499,0.0,0.0,0.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.282926,0.263492,0.260648,0.248446,0.230717,0.230373,0.228871,-1.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.188717,0.179073,0.151745,0.146303,0.135194,0.132056,0.129261,-1.0,0.0,0.0
4,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.450056,0.424316,0.422547,0.414055,0.398026,0.359367,0.339476,-1.0,0.0,0.0


In [266]:
Train_df[36]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       1.0
29       0.0
        ... 
11660    0.0
11661    0.0
11662    0.0
11663    1.0
11664    1.0
11665    0.0
11666    0.0
11667    0.0
11668    0.0
11669    0.0
11670    1.0
11671    1.0
11672    0.0
11673    0.0
11674    0.0
11675    0.0
11676    0.0
11677    0.0
11678    0.0
11679    0.0
11680    0.0
11681    0.0
11682    0.0
11683    0.0
11684    0.0
11685    0.0
11686    0.0
11687    0.0
11688    0.0
11689    0.0
Name: 36, Length: 11690, dtype: float64

In [338]:
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=5)

In [339]:
clf.fit(X_train, y_train, epochs=20)

Trial 5 Complete [00h 00m 20s]
val_accuracy: 0.8517718315124512

Best val_accuracy So Far: 0.8517718315124512
Total elapsed time: 00h 01m 27s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20
366/366 [==============================] - 1s 2ms/step - loss: 0.6339 - accuracy: 0.6887
Epoch 2/20
366/366 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7407
Epoch 3/20
366/366 [==============================] - 1s 2ms/step - loss: 0.4903 - accuracy: 0.7815
Epoch 4/20
366/366 [========

In [340]:
X_test.shape

(16627, 36)

In [341]:
predicted_y = clf.predict(X_test)

In [342]:
predicted_y_train = clf.predict(X_train)

In [343]:
f1_score(predicted_y_train, y_train)

0.7126715092816788

In [344]:
predicted_y = predicted_y.reshape(-1, 1)

In [345]:
predicted_y = predicted_y.astype(int)

In [ ]:
predicted_y = np.array(predicted_y).ravel()

In [333]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(predicted_y[i]) + str("\n"))